
This notebook implements functions to preprocess the raw data and prepare data
for analyzing temporal trends in the usage of GPT across different factors:

1. Author Frequency 
2. Number of Authors
3. Overall Trend

This process takes approximately 1 hour to complete. 

**Note** that the parquet files are  generated from a saved pkl instance of ArticleCollection. The details on this can be found at main_generate.py and scripts/generate_data.sh. The latter is a bash script I used to generate the parquet files. 

In [1]:
from base import * 
import pandas as pd 
import os 
import pickle 
from pathlib import Path 
from src.estimation import estimate_text_distribution
from src.MLE import MLE
import warnings
import numpy as np 
import matplotlib.pyplot as plt 
warnings.filterwarnings("ignore")

/Users/sawal/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/sawal/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
## the key variables 

subject = "education"
base_loc = f"inference_data/{subject}"
years = [2024, 2023, 2022, 2021]

## Author Frequency

We bin the articles for each quarter into two bins: 

    1. Written by authors with fewer than two publications in that month 
    2. Written by authors with two or more publications in that month 

In [8]:
def categorize_by_authors(collection):
    """
    maps authors to their corresponding articles  
    
    Args:
        collection: (base.ArticleCollection)
    Returns:
        (dict) (str) author name -> (List[int]) List of article ids
    """
    
    author_dict = {}
    count = 0 
    for ar_id in collection.all_articles:
        count += 1
        ar = collection.all_articles[ar_id]
        if ar.authors is not None and len(ar.authors) > 0:
            # all author data 
            #for author in ar.authors:
            #    if author not in author_dict:
            #        author_dict[author] = []
            #    author_dict[author].append(ar_id)
            
            ## first author data 
            author = ar.authors[0]
            if author not in author_dict:
                author_dict[author] = []
            author_dict[author].append(ar_id)

    return author_dict

def bin_by_freq(data_dict, cutoff=2):
    """
    bins the data into two groups according to the cutoff
    
    Args:
        cutoff: (int) 
        data_dict: (dict) (object) -> List[object]
    """
    
    cutoff_left = []
    cutoff_right = []
    for key in data_dict:
        if len(data_dict[key]) <= cutoff:
            cutoff_left += data_dict[key]
        else:
            cutoff_right += data_dict[key]
            
    return set(cutoff_left), set(cutoff_right)

def add_to_sub_collection(sub_collection, main_collection, ids=None):
    """
    add selected articles from the main collection to sub-collection
    
    Args:
        sub_collection: (ArticleCollection)
        main_collection: (ArticleCollection) 
        ids: (List[object]) list of ids of articles of interest 
    """
    if ids is None:
        ids = list(main_collection.all_articles.keys())
    for id_ in ids:
        sub_collection.add_article(main_collection.get_article(id_), id_)

def combine_collections(coll_list):
    """
    combine the a group of collections into a single collection, and returns the collection
    """
    if coll_list is None:
        raise ValueError("the parameter is None")
    if len(coll_list) == 0:
        raise ValueError("empty collection list")
        
    sample = coll_list[0]
    collection = ArticleCollection(subject=sample.subject)
    for coll in coll_list:
        add_to_sub_collection(collection, coll)
        
    return collection 
    

In [10]:
thresh = 2 # equals the value in Liang et al.
dest = Path("inference_data/parquet_files/first_author_quarterly/{}".format(subject))
dest.mkdir(parents=True, exist_ok=True)
time_window = 3
for year in [2024, 2023, 2022, 2021]:
    dest_year = dest / str(year)
    dest_year.mkdir(parents=True, exist_ok=True)
    quarterly_data = []
    months = np.arange(1, 13)
    q_count = 0
    for m in months:
        full_path = f"{base_loc}/{year}/{subject}_{year}_{m}.pkl"
        with open(full_path, "rb") as f:
            data = pickle.load(f)
            quarterly_data.append(data) 
            if len(quarterly_data) == time_window:
                combined = combine_collections(quarterly_data)
                quarterly_data = []
                by_author = categorize_by_authors(combined)
                q_count += 1
                name = f"{subject}_{year}_{q_count}"
                if len(by_author) > 0:
                    group1, group2 = bin_by_freq(by_author)
                    collection1 = ArticleCollection(subject, year)
                    collection2 = ArticleCollection(subject, year)
                    add_to_sub_collection(collection1, combined, group1)
                    add_to_sub_collection(collection2, combined, group2)
                    collection1.export_parquet(dest_year, f"{name}_less")
                    collection2.export_parquet(dest_year, f"{name}_more")

100%|███████████████████████████████████████| 7242/7242 [03:05<00:00, 39.13it/s]


File Exported as: inference_data/parquet_files/first_author_quarterly/education/2024/education_2024_1_less.parquet


100%|█████████████████████████████████████████| 362/362 [00:06<00:00, 59.74it/s]


File Exported as: inference_data/parquet_files/first_author_quarterly/education/2024/education_2024_1_more.parquet


100%|███████████████████████████████████████| 1063/1063 [00:28<00:00, 37.60it/s]


...

File Exported as: inference_data/parquet_files/first_author_quarterly/education/2021/education_2021_3_more.parquet


100%|███████████████████████████████████████| 6982/6982 [03:42<00:00, 31.34it/s]


File Exported as: inference_data/parquet_files/first_author_quarterly/education/2021/education_2021_4_less.parquet


100%|█████████████████████████████████████████| 728/728 [00:19<00:00, 37.44it/s]


File Exported as: inference_data/parquet_files/first_author_quarterly/education/2021/education_2021_4_more.parquet


## Number of Authors

We consider the number of authors involved in the article. To decide the threshold for binning, we do some exploratory analyis. 

In [11]:
def get_num_authors(article_coll):
    """
    returns the number of authors in each article contained in the input collection 
    
    Args:
        article_coll: (ArticleCollection)
        
    Returns:
        (List[int]) number of authors in each article 
    """

    num_authors = []
    for ar_id in article_coll.all_articles:
        ar = article_coll.all_articles[ar_id]
        authors =  ar.authors 
        if authors is not None:
            num_authors.append(len(authors))

    return num_authors


all_authors = []
for y in years:
    with open("inference_data/education/{}/{}_{}_whole.pkl".format(y, subject, y), "rb") as f:
        data = pickle.load(f)
    all_authors += get_num_authors(data)
all_authors = np.asarray(all_authors)
md = np.median(all_authors)
print("Median: {}".format(md))

Median: 2.0


As seen from above cell, the median number of authors is 2. So we will have two bins, 

    1. Articles with number of authors less than or equal to 2 
    2. Articles with number of authors more than 2

In [12]:
def combine_collections(coll_list):
    """
    combine the a group of collections into a single collection
    """
    
    if coll_list is None:
        raise ValueError("the parameter is None")
    if len(coll_list) == 0:
        raise ValueError("empty collection list")
    elif len(coll_list) == 1:
        return coll_list[0]
        
    sample = coll_list[0]
    collection = ArticleCollection(subject=sample.subject)
    for coll in coll_list:
        add_to_sub_collection(collection, coll)
        
    return collection 

def group_by_num_authors(collection, thresh=2):
    """
    divides the article collection into two groups: 
       1. articles with authors less than or equal to the thresh
       2. articles with authors more than the thresh 
    
    Args:
       collection: (ArticleCollection)
       thres: (int) the threshold
    
    Returns:
        (ArticleCollection, ArticleCollection) the collection of articles in the two bins 
    """
    
    collection1 = ArticleCollection(collection.subject, collection.year)
    collection2 = ArticleCollection(collection.subject, collection.year)

    for ar_id in collection.all_articles:
        ar = collection.all_articles[ar_id]
        authors = ar.authors 
        n = len(authors)
        if authors is not None and n > 0:
            if n <= thresh:
                collection1.add_article(ar, ar_id) 
            else:
                collection2.add_article(ar, ar_id)
                
    return collection1, collection2
    

In [13]:
thresh = 2 
dest = Path("inference_data/parquet_files/num_authors/{}".format(subject))
dest.mkdir(parents=True, exist_ok=True)
time_window = 1 # the time frame over which we look 
for year in [2024, 2023, 2022, 2021]:
    dest_year = dest / str(year) 
    dest_year.mkdir(parents=True, exist_ok=True) 
    months = np.arange(1, 13) 
    grouped_data = []
    q_count = 0
    for m in months:
        full_path = f"{base_loc}/{year}/{subject}_{year}_{m}.pkl"
        with open(full_path, "rb") as f:
            data = pickle.load(f)
        grouped_data.append(data)
        if len(grouped_data) == time_window:
            combined = combine_collections(grouped_data)
            sub_coll1, sub_coll2 = group_by_num_authors(combined)
            grouped_data = []
            q_count += 1
            name = f"{subject}_{year}_{q_count}"
            print("Year: {}, Month: {}, total: {}, less than {}: {}, more than {}:{}".format(
                            year, m, combined.get_size(), thresh, sub_coll1.get_size(), 
                            thresh, sub_coll2.get_size()))
            sub_coll1.export_parquet(dest_year, f"{name}_less")
            sub_coll2.export_parquet(dest_year, f"{name}_more")

Year: 2024, Month: 1, total: 2726, less than 2: 1604, more than 2:1108


100%|███████████████████████████████████████| 1604/1604 [00:38<00:00, 41.59it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2024/education_2024_1_less.parquet


100%|███████████████████████████████████████| 1108/1108 [00:33<00:00, 32.89it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2024/education_2024_1_more.parquet
Year: 2024, Month: 2, total: 2192, less than 2: 1425, more than 2:754


...

File Exported as: inference_data/parquet_files/num_authors/education/2021/education_2021_11_more.parquet
Year: 2021, Month: 12, total: 3704, less than 2: 2289, more than 2:1401


100%|███████████████████████████████████████| 2289/2289 [01:19<00:00, 28.71it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2021/education_2021_12_less.parquet


100%|███████████████████████████████████████| 1401/1401 [00:53<00:00, 26.22it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2021/education_2021_12_more.parquet


## Overall Trend 

The overall trend in the usage of GPT in scientific writing

In [14]:
for year in years:
    dest_year = dest / str(year)
    dest_year.mkdir(parents=True, exist_ok=True)
    for file in os.listdir(f"{base_loc}/{year}"):
        if "whole" not in file:
            full_path = f"{base_loc}/{year}/{file}"
            with open(full_path, "rb") as f:
                data = pickle.load(f)
                name = file.strip(".pkl")
                data.export_parquet(dest_year, name)

100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 20.08it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2024/education_2024_5.parquet


100%|███████████████████████████████████████| 1042/1042 [00:32<00:00, 31.70it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2024/education_2024_4.parquet


100%|███████████████████████████████████████████| 24/24 [00:01<00:00, 23.56it/s]


...

File Exported as: inference_data/parquet_files/num_authors/education/2021/education_2021_10.parquet


100%|███████████████████████████████████████| 1935/1935 [00:57<00:00, 33.68it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2021/education_2021_8.parquet


100%|███████████████████████████████████████| 2044/2044 [01:01<00:00, 33.05it/s]


File Exported as: inference_data/parquet_files/num_authors/education/2021/education_2021_9.parquet
